<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Quick-Check-of-the-DataFrame-Structure" data-toc-modified-id="Quick-Check-of-the-DataFrame-Structure-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Quick Check of the DataFrame Structure</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# The prefixes of pitch types below 'Sweeper' are made up by me 
# due to leak of corresponding columns in the main table.
pitch_type_mapping = {'4-Seamer':'ff',
                      '4-SeamFastball':'ff',
                      'Split-Finger':'fs',
                     'Changeup':'ch',
                     'Slider':'sl',
                     'Sinker':'si',
                     'Curveball':'cu',
                     'Splitter':'fs',
                     'Cutter':'fc',
                     'Knuckleball':'kn',
                     'Sweeper':'sw',
                     'Slurve':'su',
                     'Forkball':'fo',
                     'Screwball':'sc'}

In [3]:
csv_path = 'MLB_Player_Stats_CSV\\'
#player_type = 'pitcher'
player_type = 'batter'
csv_new_filename = f'{csv_path}pitch_type_{player_type}.csv'
years = [*range(2017,2024)]
#print(years)

In [4]:
PA_df_raw = pd.read_csv(csv_new_filename)
# try:
#     PA_df_raw = PA_df_raw.drop(['Unnamed: 0'], axis=1)
# except:
#     pass
player_ids = list(set(PA_df_raw['player_id']))
try:
    player_ids.remove(519381)
except:
    pass
print(PA_df_raw.shape)
print(PA_df_raw.columns)
print(len(player_ids))

(41997, 20)
Index(['player_id', 'player_name', 'Year', 'Pitch Type', 'Team', 'RV/100',
       'Run Value', 'Pitches', '%', 'PA', 'BA', 'SLG', 'wOBA', 'Whiff%', 'K%',
       'PutAway %', 'xBA', 'xSLG', 'xwOBA', 'Hard Hit %'],
      dtype='object')
2088


In [5]:
player_id = player_ids[19]
df_this_player = PA_df_raw[PA_df_raw.player_id.eq(player_id)]
years_this_player = list(set(df_this_player['Year']).intersection(set(years)))
df_this_player

,player_id,player_name,Year,Pitch Type,Team,RV/100,Run Value,Pitches,%,PA,BA,SLG,wOBA,Whiff%,K%,PutAway %,xBA,xSLG,xwOBA,Hard Hit %
294,622668,"Castillo, Ivan",2021,Sinker,NaN,-8.2,-1.0,7,31.8,1.0,0.0,0.0,0.00,0.0,0.0,NaN,0.083,0.109,0.085,0.0
295,622668,"Castillo, Ivan",2021,Slider,NaN,4.5,0.0,5,22.7,NaN,NaN,NaN,NaN,50.0,NaN,0.0,NaN,NaN,NaN,NaN
296,622668,"Castillo, Ivan",2021,4-SeamFastball,NaN,0.4,0.0,5,22.7,2.0,0.0,0.0,0.35,0.0,0.0,0.0,0.051,0.051,0.373,0.0
297,622668,"Castillo, Ivan",2021,Curveball,NaN,20.9,1.0,5,22.7,1.0,1.0,1.0,0.90,0.0,0.0,0.0,0.449,0.498,0.408,100.0


In [6]:
if 'new_df' in globals():
    del new_df
if 'this_new_df' in globals():
    del this_new_df
for i, player_id in enumerate(player_ids):
    print(f'{i+1}/{len(player_ids)} player_id: {player_id}')
    df_this_player = PA_df_raw[PA_df_raw.player_id.eq(player_id)]
    years_this_player = list(set(df_this_player['Year']).intersection(set(years)))
    for yr in years_this_player:
        df_this_year = df_this_player[df_this_player.Year.eq(yr)].reset_index(drop=True)
        dfs_tobe_concatenated = []
        for index, row in df_this_year.iterrows():
            if type(row['Pitch Type']) == str:
                PT_prefix = pitch_type_mapping[row['Pitch Type']]
                if player_type == 'pitcher':
                    row_short = row.drop(['player_id', 'player_name', 'Year',
                                          'Team', 'Hand', 'MPH', 'Inches of Drop','Drop vs Avg',
                                          '% vs Avg', 'Inches of Break', 'Break vs Avg',
                                          '% Break vs Avg', 'Pitch Type', '%', 'Pitches', 'Run Value',
                                         'xBA','xSLG','xwOBA'], axis=0)
                elif player_type == 'batter':
                    row_short = row.drop(['player_id', 'player_name', 'Year',
                                          'Team', 'Pitch Type', 'Pitches', 'Run Value',
                                         'xBA','xSLG','xwOBA'], axis=0)
                #print(row_short)
                this_dict = {'player_id':[player_id],'year':[yr]}
                for col in row_short.keys():
                    new_col = PT_prefix + '_' + col
                    this_dict[new_col] = [row_short[col]]
                #print(this_dict)
                this_df = pd.DataFrame(this_dict)
                if len(dfs_tobe_concatenated) > 0:
                    this_df = this_df.drop(['year', 'player_id'], axis=1)
                dfs_tobe_concatenated.append(this_df)
                #print(this_df)
                
#                 if not 'this_new_df' in globals():
#                     this_new_df = this_df
#                 else:
#                     pause
#                     for key in list(this_dict.keys())[2::]:
#                         this_new_df.insert(this_new_df.shape[1], key, this_dict[key])
        this_new_df = pd.concat(dfs_tobe_concatenated, axis=1)
        if 'this_new_df' in globals():
            if not 'new_df' in globals():
                new_df = this_new_df
            else:
                this_new_col = this_new_df.columns.tolist()
                new_col = new_df.columns.tolist()
                diff_col = list(set(this_new_col).difference(set(new_col)))
                new_df = pd.concat([this_new_df, new_df], ignore_index=True)
#                 try:
#                     new_df = pd.concat([this_new_df, new_df], ignore_index=True)
#                 except:
#                     pause
            del this_new_df

1/2088 player_id: 663554
2/2088 player_id: 466948
3/2088 player_id: 606213
4/2088 player_id: 663559
5/2088 player_id: 663567
6/2088 player_id: 622608
7/2088 player_id: 663574
8/2088 player_id: 663586
9/2088 player_id: 475174
10/2088 player_id: 466988
11/2088 player_id: 671790
12/2088 player_id: 663609
13/2088 player_id: 663611
14/2088 player_id: 467008
15/2088 player_id: 606273
16/2088 player_id: 663616
17/2088 player_id: 622663
18/2088 player_id: 663624
19/2088 player_id: 622666
20/2088 player_id: 622668
21/2088 player_id: 663630
22/2088 player_id: 450641
23/2088 player_id: 622682
24/2088 player_id: 606299
25/2088 player_id: 663647
26/2088 player_id: 622694
27/2088 player_id: 663656
28/2088 player_id: 450665
29/2088 player_id: 475243
30/2088 player_id: 663662
31/2088 player_id: 467055
32/2088 player_id: 475247
33/2088 player_id: 475253
34/2088 player_id: 622713
35/2088 player_id: 606336
36/2088 player_id: 647304
37/2088 player_id: 663697
38/2088 player_id: 663698
39/2088 player_id: 64

313/2088 player_id: 656582
314/2088 player_id: 664774
315/2088 player_id: 664789
316/2088 player_id: 656605
317/2088 player_id: 607455
318/2088 player_id: 607457
319/2088 player_id: 607461
320/2088 player_id: 607468
321/2088 player_id: 607471
322/2088 player_id: 607473
323/2088 player_id: 656627
324/2088 player_id: 656629
325/2088 player_id: 517369
326/2088 player_id: 460026
327/2088 player_id: 492802
328/2088 player_id: 664842
329/2088 player_id: 664848
330/2088 player_id: 615698
331/2088 player_id: 664850
332/2088 player_id: 615699
333/2088 player_id: 664854
334/2088 player_id: 460060
335/2088 player_id: 656669
336/2088 player_id: 476451
337/2088 player_id: 664869
338/2088 player_id: 476454
339/2088 player_id: 517414
340/2088 player_id: 623912
341/2088 player_id: 492841
342/2088 player_id: 664874
343/2088 player_id: 460075
344/2088 player_id: 460077
345/2088 player_id: 656686
346/2088 player_id: 607536
347/2088 player_id: 460086
348/2088 player_id: 656695
349/2088 player_id: 435522
3

618/2088 player_id: 592169
619/2088 player_id: 657709
620/2088 player_id: 543022
621/2088 player_id: 592178
622/2088 player_id: 518452
623/2088 player_id: 608566
624/2088 player_id: 543031
625/2088 player_id: 641341
626/2088 player_id: 543037
627/2088 player_id: 543038
628/2088 player_id: 641343
629/2088 player_id: 592192
630/2088 player_id: 665922
631/2088 player_id: 608577
632/2088 player_id: 518468
633/2088 player_id: 502082
634/2088 player_id: 518466
635/2088 player_id: 665923
636/2088 player_id: 543045
637/2088 player_id: 665926
638/2088 player_id: 592200
639/2088 player_id: 641355
640/2088 player_id: 543054
641/2088 player_id: 592206
642/2088 player_id: 543056
643/2088 player_id: 641360
644/2088 player_id: 657746
645/2088 player_id: 502100
646/2088 player_id: 608596
647/2088 player_id: 649557
648/2088 player_id: 543063
649/2088 player_id: 665947
650/2088 player_id: 543068
651/2088 player_id: 657757
652/2088 player_id: 502110
653/2088 player_id: 592229
654/2088 player_id: 502117
6

926/2088 player_id: 641857
927/2088 player_id: 502593
928/2088 player_id: 543557
929/2088 player_id: 592710
930/2088 player_id: 691016
931/2088 player_id: 592716
932/2088 player_id: 592717
933/2088 player_id: 682829
934/2088 player_id: 691023
935/2088 player_id: 691026
936/2088 player_id: 641878
937/2088 player_id: 592731
938/2088 player_id: 445276
939/2088 player_id: 502624
940/2088 player_id: 666464
941/2088 player_id: 682848
942/2088 player_id: 519008
943/2088 player_id: 592741
944/2088 player_id: 543590
945/2088 player_id: 592743
946/2088 player_id: 543592
947/2088 player_id: 625510
948/2088 player_id: 543594
949/2088 player_id: 519023
950/2088 player_id: 519025
951/2088 player_id: 641907
952/2088 player_id: 543606
953/2088 player_id: 592761
954/2088 player_id: 641914
955/2088 player_id: 543611
956/2088 player_id: 519037
957/2088 player_id: 592767
958/2088 player_id: 592769
959/2088 player_id: 519043
960/2088 player_id: 641924
961/2088 player_id: 641927
962/2088 player_id: 519048
9

1230/2088 player_id: 446653
1231/2088 player_id: 643265
1232/2088 player_id: 643269
1233/2088 player_id: 643271
1234/2088 player_id: 643275
1235/2088 player_id: 676050
1236/2088 player_id: 643289
1237/2088 player_id: 643290
1238/2088 player_id: 676059
1239/2088 player_id: 676070
1240/2088 player_id: 643327
1241/2088 player_id: 643335
1242/2088 player_id: 643338
1243/2088 player_id: 643348
1244/2088 player_id: 676116
1245/2088 player_id: 520471
1246/2088 player_id: 643354
1247/2088 player_id: 643376
1248/2088 player_id: 643377
1249/2088 player_id: 643393
1250/2088 player_id: 643396
1251/2088 player_id: 643410
1252/2088 player_id: 643414
1253/2088 player_id: 643418
1254/2088 player_id: 545121
1255/2088 player_id: 528748
1256/2088 player_id: 643436
1257/2088 player_id: 643446
1258/2088 player_id: 676220
1259/2088 player_id: 594311
1260/2088 player_id: 643466
1261/2088 player_id: 446868
1262/2088 player_id: 643493
1263/2088 player_id: 676265
1264/2088 player_id: 455104
1265/2088 player_id:

1534/2088 player_id: 628451
1535/2088 player_id: 628452
1536/2088 player_id: 669432
1537/2088 player_id: 579328
1538/2088 player_id: 669450
1539/2088 player_id: 489232
1540/2088 player_id: 677649
1541/2088 player_id: 669456
1542/2088 player_id: 677651
1543/2088 player_id: 661269
1544/2088 player_id: 448281
1545/2088 player_id: 669477
1546/2088 player_id: 595751
1547/2088 player_id: 456488
1548/2088 player_id: 489265
1549/2088 player_id: 448306
1550/2088 player_id: 489267
1551/2088 player_id: 456501
1552/2088 player_id: 595777
1553/2088 player_id: 595798
1554/2088 player_id: 489334
1555/2088 player_id: 661388
1556/2088 player_id: 112526
1557/2088 player_id: 620439
1558/2088 player_id: 620443
1559/2088 player_id: 620446
1560/2088 player_id: 620453
1561/2088 player_id: 620454
1562/2088 player_id: 595879
1563/2088 player_id: 677800
1564/2088 player_id: 595881
1565/2088 player_id: 595885
1566/2088 player_id: 669622
1567/2088 player_id: 595909
1568/2088 player_id: 595918
1569/2088 player_id:

1829/2088 player_id: 621439
1830/2088 player_id: 621446
1831/2088 player_id: 621450
1832/2088 player_id: 621453
1833/2088 player_id: 621458
1834/2088 player_id: 621466
1835/2088 player_id: 400284
1836/2088 player_id: 621471
1837/2088 player_id: 670623
1838/2088 player_id: 547749
1839/2088 player_id: 621493
1840/2088 player_id: 605113
1841/2088 player_id: 621500
1842/2088 player_id: 605119
1843/2088 player_id: 605125
1844/2088 player_id: 621512
1845/2088 player_id: 572362
1846/2088 player_id: 605131
1847/2088 player_id: 621514
1848/2088 player_id: 572365
1849/2088 player_id: 588751
1850/2088 player_id: 605135
1851/2088 player_id: 605137
1852/2088 player_id: 605141
1853/2088 player_id: 621529
1854/2088 player_id: 621532
1855/2088 player_id: 605151
1856/2088 player_id: 678882
1857/2088 player_id: 605154
1858/2088 player_id: 605156
1859/2088 player_id: 457705
1860/2088 player_id: 457706
1861/2088 player_id: 621545
1862/2088 player_id: 457708
1863/2088 player_id: 605164
1864/2088 player_id:

###### Quick Check of the DataFrame Structure

In [7]:
whiff_col_list = ['year']
for col in new_df.columns:
    if 'Whiff%' in col:
        whiff_col_list.append(col)
print(whiff_col_list)

['year', 'ff_Whiff%', 'ch_Whiff%', 'cu_Whiff%', 'sl_Whiff%', 'si_Whiff%', 'fs_Whiff%', 'fc_Whiff%', 'sw_Whiff%', 'su_Whiff%', 'kn_Whiff%', 'sc_Whiff%']


In [8]:
new_df

,player_id,year,ff_RV/100,ff_%,ff_PA,ff_BA,ff_SLG,ff_wOBA,ff_Whiff%,ff_K%,...,sc_RV/100,sc_%,sc_PA,sc_BA,sc_SLG,sc_wOBA,sc_Whiff%,sc_K%,sc_PutAway %,sc_Hard Hit %
0,663550,2023,-1.9,35.3,21.0,0.222,0.444,0.307,47.9,47.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,671739,2023,-0.6,34.2,155.0,0.319,0.435,0.365,19.3,15.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,671739,2022,2.1,33.3,134.0,0.369,0.697,0.473,25.0,18.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,671732,2023,-2.7,28.6,40.0,0.189,0.270,0.220,20.5,17.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,663538,2023,1.5,26.4,201.0,0.307,0.470,0.404,6.6,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5898,606213,2022,-1.2,40.8,27.0,0.208,0.375,0.298,24.0,25.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5899,606213,2021,0.0,34.4,73.0,0.238,0.397,0.358,24.4,20.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5900,606213,2020,-1.2,66.7,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5901,466948,2017,-3.5,100.0,1.0,0.000,0.000,0.000,50.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
PA_df_raw[PA_df_raw.player_id.eq(player_ids[562])][['Year', 'Pitch Type', 'Whiff%']]

,Year,Pitch Type,Whiff%
11163,2021,4-SeamFastball,56.4
11164,2021,Sinker,62.5
11165,2021,Curveball,71.4
11166,2021,Cutter,25.0
11167,2021,Slider,71.4
11168,2021,Changeup,100.0
11169,2021,Slurve,100.0
11170,2021,Split-Finger,100.0
11171,2019,4-SeamFastball,44.4
11172,2019,Sinker,0.0


In [10]:
for year in years:
    #----Define two dfs to be merged----------
    old_df = pd.read_csv(f'{csv_path}{year}_{player_type}_arranged.csv').\
    sort_values('player_id').reset_index(drop=True)
    new_df_this_year = new_df[new_df.year.eq(year)].\
    sort_values('player_id').reset_index(drop=True)
    #----Extract players in both of the dfs
    old_id = set(old_df['player_id']); new_id = set(new_df_this_year['player_id'])
    common_id = list(old_id.intersection(new_id))
    #----Check visually
    print('=============================')
    print(year)
    print(len(old_id))
    print(len(new_id))
    print(len(common_id))
    #----merge dfs
    old_df_2 = old_df[old_df.player_id.isin(common_id)].reset_index(drop=True)
    new_df_this_year_2 = new_df_this_year[new_df_this_year.player_id.isin(common_id)].reset_index(drop=True)
    new_df_this_year_2 = new_df_this_year_2.drop(['year'], axis=1)
    combined_df = pd.merge(old_df_2, new_df_this_year_2)
    #-----ReOrder few columns
    cols = list(combined_df.columns)
    cols.remove('player_age')
    cols.insert(3, 'player_age')
    cols.remove('pa')
    cols.insert(4, 'pa')
    if player_type == 'pitcher':
        cols.remove('name_abbrev')
        cols.insert(5, 'name_abbrev')
        cols.remove('pitch_hand')
        cols.insert(6, 'pitch_hand')
    combined_df = combined_df.reindex(columns=cols)
    #----save csv file
    output_filename = f'{csv_path}{year}_{player_type}_arranged_V2.csv'
    combined_df.to_csv(output_filename)

2017
957
956
956
2018
990
988
988
2019
990
987
987
2020
581
581
580
2021
1049
1045
1045
2022
693
691
690
2023
656
655
654
